In [31]:
# 5.4 データ拡張まで

In [32]:
from pathlib import Path
import shutil
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [33]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

C:\Users\210450279\Anaconda3\envs\rebases\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [34]:
# 再現性の確保のために乱数シードを固定（数値は何でもよい）
tf.set_random_seed(12345)

In [35]:
# 入力データ
# MNISTのワンホット表現での読み込み
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True, validation_size=10000)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [36]:
# 0 入力画像
x = tf.placeholder(tf.float32, name='x')

In [37]:
# 1 サイズ変更
x_1 = tf.reshape(x, [-1, 28, 28, 1])

In [38]:
# 2 畳み込み
# ランダムカーネル
k_0 = tf.Variable(tf.truncated_normal([4, 4, 1, 10], mean=0.0, stddev=0.1))
# 畳み込み
x_2 = tf.nn.conv2d(x_1, k_0, strides=[1, 3, 3, 1], padding='VALID')

In [39]:
# 3 活性化関数
x_3 = tf.nn.relu(x_2)

In [40]:
# 4 プーリング
x_4 = tf.nn.max_pool(x_3, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')


In [41]:
# 5 サイズ変更
x_5 = tf.reshape(x_4, [-1, 160])

In [42]:
#ドロップアウト付きの全結合
def matmul_plus_bias_with_dropout(x, w, b, p):
    return tf.matmul(tf.nn.dropout(x, keep_prob=p), w) + b

In [43]:
# 6 全結合
# 重みとバイアス
w_1 = tf.Variable(tf.zeros([160, 40]))
b_1 = tf.Variable([0.1] * 40)
#ドロップアウト率
p_1 = tf.placeholder(tf.float32, name='p_1')
# 全結合
x_6 = matmul_plus_bias_with_dropout(x_5, w_1, b_1, p_1)

In [44]:
# 7 活性化関数
x_7 = tf.nn.relu(x_6)

In [45]:
# 8 全結合
# 重みとバイアス
w_2 = tf.Variable(tf.zeros([40, 10]))
b_2 = tf.Variable([0.1] * 10)
#ドロップアウト率
p_2 = tf.placeholder(tf.float32, name='p_2')
# 全結合
x_8 = matmul_plus_bias_with_dropout(x_7, w_2, b_2, p_2)

In [46]:
# 9 確率化
y = tf.nn.softmax(x_8, name='y')

In [47]:
# 10 損失関数の最小化
# 正解ラベル
labels = tf.placeholder(tf.float32, name='labels')
# 損失関数（交差エントロピー）と最適化処理（Adam）
loss = -tf.reduce_sum(labels * tf.log(y + 1e-9)) / tf.cast(tf.shape(y)[0], tf.float32)
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [48]:
# 11 精度検証 (Top-1 accuracy)
prediction_match = tf.equal(tf.argmax(y, axis=1), tf.argmax(labels, axis=1))
accuracy = tf.reduce_mean(tf.cast(prediction_match, tf.float32), name='accuracy')

In [49]:
# 損失関数と精度の出力をトレース対象とする
loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar('accuracy', accuracy)
summary = tf.summary.merge([loss_summary, accuracy_summary])

image_summary_op = tf.summary.image('input', x)

In [50]:
# 回転
def random_rotate_image(image):
    image = ndimage.rotate(image, np.random.uniform(-5, 5), reshape=False)
    return image

# 平行移動
def random_shift_image(image):
    shift = [*np.random.randint(-3, 3, 2), 0]
    image = ndimage.interpolation.shift(image, shift)
    return image

# # データ拡張
def augment(images):
    # 画像の変形を行うには、事前に画像の形状変換が必要である (784,) → (28,28,1)
    # バリデーションセットやテストセットには、データ拡張は行わない。
    images = images.reshape((-1, 28, 28, 1))
    preprocessed_images = []  # コピー画像データセットを入れるための空リスト
    for image in images:
        # データセットの画像自体を書き換えないよう、画像をコピーしてからデータ拡張を適用する
        preprocessed_image = image.copy()
        preprocessed_image = random_rotate_image(preprocessed_image)
        preprocessed_image = random_shift_image(preprocessed_image)
        preprocessed_images.append(preprocessed_image)
    return preprocessed_images

# MinMaxScaler: 正規化（最大1, 最小0）
def preprocess(images):
    scaler = MinMaxScaler()
    return scaler.partial_fit(images)

In [51]:
# パラメーター
# バッチサイズ
BATCH_SIZE = 32
# 学習回数
NUM_TRAIN = 10000
# 学習中の出力頻度
OUTPUT_BY = 500
#ドロップアウト率
DROPOUT_PROB_1 = 0.2
DROPOUT_PROB_2 = 0.5

In [52]:
# 以前のログが残っていれば削除する（通常は残すようにしてください）
logdir = Path('logs/log2')
if logdir.exists():
    shutil.rmtree(str(logdir))

In [53]:
# 学習の実行
# TensorBoard上でグラフを重ねて描くために、2つのwriterを使用する
with tf.summary.FileWriter('logs/log2/train') as train_writer, \
    tf.summary.FileWriter('logs/log2/val') as val_writer:
    sess.run(tf.global_variables_initializer())
    dropout_prob = {p_1: DROPOUT_PROB_1, p_2: DROPOUT_PROB_2}
    saver = tf.train.Saver()
    for i in range(NUM_TRAIN):
        batch = mnist.train.next_batch(BATCH_SIZE)
        inout = {x: augment(batch[0]), labels: batch[1]} # augment()でデータ拡張を行っている
        if i % OUTPUT_BY == 0:
            train_loss, train_accuracy, train_summary, image_summary = \
                sess.run([loss, accuracy, summary, image_summary_op], feed_dict={**inout, p_1: 1.0, p_2: 1.0})
            val_loss, val_accuracy, val_summary = \
                sess.run([loss, accuracy, summary], feed_dict={
                x: mnist.validation.images, labels: mnist.validation.labels, p_1: 1.0, p_2: 1.0})
            print('step {:d}, train_loss {:.2f}, val_loss {:.2f}, train_acc {:.2f}, val_acc {:.2f}'.format(
                i, train_loss, val_loss, train_accuracy, val_accuracy))
            # ログの出力
            train_writer.add_summary(train_summary, global_step=i)
            val_writer.add_summary(val_summary, global_step=i)
            train_writer.add_summary(image_summary, global_step=i)
            # 過程の保存
            saver.save(sess, 'model2/my-model', global_step=i)
            # ファイルへの強制書き出し
            train_writer.flush()
            val_writer.flush()
        optimizer.run(feed_dict={**inout, **dropout_prob})

# 最終結果の保存
saver.save(sess, 'model2/my-model')

step 0, train_loss 2.30, val_loss 2.30, train_acc 0.12, val_acc 0.10
step 500, train_loss 1.86, val_loss 1.70, train_acc 0.38, val_acc 0.52
step 1000, train_loss 1.74, val_loss 1.56, train_acc 0.53, val_acc 0.59
step 1500, train_loss 1.49, val_loss 1.41, train_acc 0.62, val_acc 0.63
step 2000, train_loss 1.47, val_loss 1.28, train_acc 0.62, val_acc 0.67
step 2500, train_loss 1.34, val_loss 1.20, train_acc 0.62, val_acc 0.74
step 3000, train_loss 1.33, val_loss 1.15, train_acc 0.59, val_acc 0.72
step 3500, train_loss 1.24, val_loss 1.08, train_acc 0.62, val_acc 0.76
step 4000, train_loss 1.03, val_loss 1.06, train_acc 0.88, val_acc 0.78
step 4500, train_loss 1.30, val_loss 1.02, train_acc 0.62, val_acc 0.78
step 5000, train_loss 1.00, val_loss 1.01, train_acc 0.81, val_acc 0.78
step 5500, train_loss 1.27, val_loss 0.98, train_acc 0.56, val_acc 0.79
step 6000, train_loss 1.19, val_loss 0.95, train_acc 0.62, val_acc 0.80
step 6500, train_loss 0.96, val_loss 0.94, train_acc 0.84, val_acc 0

'model2/my-model'

In [56]:
# テストデータによる精度検証
test_accuracy = accuracy.eval(feed_dict={x : mnist.test.images,\
    labels : mnist.test.labels, p_1 : 1.0, p_2 :1.0})
print('test_accuracy{:.2f}'.format(test_accuracy))

test_accuracy0.82


In [57]:
# 上記はテンソルを用いて正解率を計算しているが、次のようにNumpy形式で正解率を計算することもできる。

# test_y は予測結果。
test_y = y.eval(feed_dict={x: mnist.test.images, p_1: 1.0, p_2: 1.0}).argmax(axis=1)

# test_labels は正解ラベル。
test_labels = mnist.test.labels.argmax(axis=1)

# 予測結果と正解ラベルから、正解率を計算する。
test_accuracy = np.sum(test_y == test_labels) / float(test_y.shape[0])
print('test accuracy {:.2f}'.format(test_accuracy))

test accuracy 0.82
